# Volume-weighted T/S Diagram from WOA

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import xarray as xr
import gsw

In [2]:
ds_salt = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA18/1_deg/annual/Avg_Decades_1981-2010/salt')
ds_temp = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA18/1_deg/annual/Avg_Decades_1981-2010/temp')
ds = xr.merge([ds_salt, ds_temp])
ds

<xarray.Dataset>
Dimensions:  (lat: 180, lev: 102, lon: 360, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-15
  * lev      (lev) float64 0.0 5.0 10.0 15.0 ... 5.2e+03 5.3e+03 5.4e+03 5.5e+03
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Data variables:
    san      (time, lev, lat, lon) float32 ...
    smn      (time, lev, lat, lon) float32 ...
    sdd      (time, lev, lat, lon) float32 ...
    ssd      (time, lev, lat, lon) float32 ...
    sse      (time, lev, lat, lon) float32 ...
    soa      (time, lev, lat, lon) float32 ...
    sgp      (time, lev, lat, lon) float32 ...
    tan      (time, lev, lat, lon) float32 ...
    tmn      (time, lev, lat, lon) float32 ...
    tdd      (time, lev, lat, lon) float32 ...
    tsd      (time, lev, lat, lon) float32 ...
    tse      (time, lev, lat, lon) float32 ...
    toa      (time, lev, lat, lon) float32 ...
    tgp      (time, lev, lat, lon) float32 ...

In [3]:
sbins = np.arange(31,38, 0.025)
tbins = np.arange(-2, 32, 0.1)

In [ ]:
sst = ds.tan[0].load()
sss = ds.san[0].load()
ds_surf = xr.merge([sst, sss])

In [ ]:
ds_surf

In [4]:
from xhistogram.xarray import histogram
from matplotlib.colors import LogNorm

In [5]:
salt_bins = np.linspace(30, 37, 100)
theta_bins = np.linspace(-2, 32, 100)
ts_hist = histogram(ds_surf.tan, ds_surf.san, bins=[theta_bins, salt_bins])
ts_hist.where(ts_hist>0).plot(norm=LogNorm())

NameError: name 'ds_surf' is not defined

In [7]:
lower_lat = -79.88
upper_lat = -59.88
left_lon = -74.88
right_lon = 30.12

In [8]:
#set the boundaries
lat_range = dict(lat=slice(lower_lat, upper_lat))
lon_range = dict(lon=slice(left_lon, right_lon))

In [9]:
temp = ds.tan.sel(**lat_range, **lon_range)
salt = ds.san.sel(**lat_range, **lon_range)

In [18]:
# Create a dz variable
dz = np.diff(ds.lev)
dz = np.insert(dz, 0, dz[0]) #inserts 1st value of original dz into the first index to make it same size as ds.lev
dz = xr.DataArray(dz, coords= {'lev':ds.lev}, dims='lev')

In [20]:
rad_earth = 6378.1e3 #meters

# Create a dy variable
dy = np.diff(ds.lat*np.pi*rad_earth/180)
dy = np.insert(dy, 0, dy[0]) #inserts 1st value of original dy into the first index to make it same size as ds.lat
dy = xr.DataArray(dy, coords= {'lat':ds.lat}, dims='lat')

In [26]:
dx = 2*np.pi*rad_earth*np.cos(ds.lat)/360

In [27]:
dx

<xarray.DataArray 'lat' (lat: 180)>
array([   3938.8214098 ,   95741.07689858,   99519.42781941,
         11800.07576042,  -86768.21153385, -105562.20529602,
        -27302.79433407,   76058.67982533,  109492.1545159 ,
         42259.04729349,  -63826.83312297, -111230.61751869,
        -56369.485134  ,   50317.49192169,  110742.79895558,
         69351.6873463 ,  -35801.04577748, -108038.46251843,
        -80945.81506483,   20568.04145862,  103171.7355194 ,
         90919.81174447,   -4923.36765012,  -96240.02553247,
        -99074.04777388,  -10819.84739537,   87382.07078017,
        105245.31606348,   26346.50312167,  -76775.16328708,
       -109310.09863649,  -41345.83340887,   64631.60037879,
        111187.03884209,   55517.62655927,  -51194.43554949,
       -110838.56970928,  -68578.23403661,   36732.61374459,
        108271.66585014,   80266.24769344,  -21535.58842584,
       -103537.70386286,  -90347.73185695,    5907.52815832,
         96731.4340288 ,   98620.90555306,    9838.77132545,
        -87989.08388497, -104920.18115399,  -25388.14773424,
         77485.63162893,  109119.47861576,   40429.38019352,
        -65431.303929  , -111134.74897112,  -54661.41833334,
         52067.36823606,  110925.6565702 ,   67799.40781357,
        -37661.30381387, -108496.38639884,  -79580.39168544,
         22501.44813977,  103895.56031602,   89768.57347664,
         -6891.22582817,  -97215.26388708,  -98160.03665937,
         -8856.92441523,   88589.20329047,  104586.82604096,
         24427.80325625,  -78190.02918767, -108920.30938824,
        -39509.759449  ,   66225.88111905,  111073.75200255,
         53800.92753776,  -52936.22158956, -111004.05271533,
        -67015.26969604,   38587.04322504,  108712.60655828,
         78888.30077571,  -23465.54492801, -104245.27684182,
        -89182.38197899,    7874.3835897 ,   97691.4772006 ,
         97691.4772006 ,    7874.3835897 ,  -89182.38197899,
       -104245.27684182,  -23465.54492801,   78888.30077571,
        108712.60655828,   38587.04322504,  -67015.26969604,
       -111004.05271533,  -52936.22158956,   53800.92753776,
        111073.75200255,   66225.88111905,  -39509.759449  ,
       -108920.30938824,  -78190.02918767,   24427.80325625,
        104586.82604096,   88589.20329047,   -8856.92441523,
        -98160.03665937,  -97215.26388708,   -6891.22582817,
         89768.57347664,  103895.56031602,   22501.44813977,
        -79580.39168544, -108496.38639884,  -37661.30381387,
         67799.40781357,  110925.6565702 ,   52067.36823606,
        -54661.41833334, -111134.74897112,  -65431.303929  ,
         40429.38019352,  109119.47861576,   77485.63162893,
        -25388.14773424, -104920.18115399,  -87989.08388497,
          9838.77132545,   98620.90555306,   96731.4340288 ,
          5907.52815832,  -90347.73185695, -103537.70386286,
        -21535.58842584,   80266.24769344,  108271.66585014,
         36732.61374459,  -68578.23403661, -110838.56970928,
        -51194.43554949,   55517.62655927,  111187.03884209,
         64631.60037879,  -41345.83340887, -109310.09863649,
        -76775.16328708,   26346.50312167,  105245.31606348,
         87382.07078017,  -10819.84739537,  -99074.04777388,
        -96240.02553247,   -4923.36765012,   90919.81174447,
        103171.7355194 ,   20568.04145862,  -80945.81506483,
       -108038.46251843,  -35801.04577748,   69351.6873463 ,
        110742.79895558,   50317.49192169,  -56369.485134  ,
       -111230.61751869,  -63826.83312297,   42259.04729349,
        109492.1545159 ,   76058.67982533,  -27302.79433407,
       -105562.20529602,  -86768.21153385,   11800.07576042,
         99519.42781941,   95741.07689858,    3938.8214098 ])
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5

In [25]:
# Create a dx variable
dx = 2*np.cos(ds.lat)*np.pi*rad_earth/360
dx = xr.DataArray(dx, coords={'lon':ds.lon}, dims='lon')

ValueError: conflicting sizes for dimension 'lon': length 180 on the data but length 360 on coordinate 'lon'

In [28]:
dV = dx * dy * dz

In [29]:
salt_bins = np.linspace(33, 35, 100)
theta_bins = np.linspace(-2.2, 4.2, 100)

delta_salt = salt_bins[1] - salt_bins[0]
delta_theta = theta_bins[1] - theta_bins[0]

ts_hist = histogram(temp, salt, 
                    bins=[theta_bins, salt_bins], 
                    weights=dV, dim=['lon', 'lat', 'lev']) / (delta_salt * delta_theta)

ts_hist.where(ts_hist>0).plot(norm=LogNorm(), figsize=(12,8), 
                              cbar_kwargs={'shrink':0.8, 'label': r'volume [$\frac{m^3}{psu ˚C}$]'})

plt.xlabel('Salinity [psu]')
plt.ylabel('Potential Temperature [˚C]')
plt.title('Volume-weighted mean histogram, T-S Diagram of Weddell Gyre \n 1981-2010 (WOA data)', fontsize=20);

ValueError: indexes along dimension 'lat' are not equal

* You can take the difference between the scatter plots (ex: SOSE-WOA) and compare does the model have enough points
* this is a naive way - there is no weights involved, you have to figure that out yourself for WOA data - refer back to notes in Slack by Ryan and maybe xgcm package by Julius

## 2005 - 2017

In [ ]:
ds_salt_0517 = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA18/1_deg/annual/2005-2017/salt')
ds_temp_0517 = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA18/1_deg/annual/2005-2017/temp')
ds_0517 = xr.merge([ds_salt_0517, ds_temp_0517])
ds_0517

In [ ]:
temp_0517 = ds_0517.tan.sel(**lat_range, **lon_range)
salt_0517 = ds_0517.san.sel(**lat_range, **lon_range)

In [ ]:
ts_hist_0517 = histogram(temp_0517, salt_0517, bins=[theta_bins, salt_bins]) / (delta_salt * delta_theta)
ts_hist_0517.where(ts_hist_0517>0).plot(norm=LogNorm(), figsize=(12,8), 
                                        cbar_kwargs={'shrink':0.8, 'label': r'volume [$\frac{m^3}{psu ˚C}$]'})

plt.xlabel('Salinity [psu]')
plt.ylabel('Potential Temperature [˚C]')
plt.title('Volume-weighted mean histogram, T-S Diagram of Weddell Gyre \n 2005-2017 (WOA data)', fontsize=20);